In [1]:
%matplotlib inline

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

**Questions**

---

Topic: Extreme weather in Midwest (North TX, OK, NE, KS, MO, IA, SD)

1) What can daily weather data reveal severe weather in the Midwest? Severe summer thunderstorms?  Extreme blizzards? Rapid temperature changes?

2) Can one identify "Tornado Season" (roughly May to June) in "Tornado Alley" based off of daily wind speed and rain data? 

3) Which areas undergo the most extreme temperature differences from day-to-day?  How is this correlated with snow, precipitation?

In [60]:
data = pd.read_csv("weather.csv")

In [61]:
data.head()

,station,state,latitude,longitude,elevation,date,TMIN,TMAX,TAVG,AWND,WDF5,WSF5,SNOW,SNWD,PRCP
0,GUAM INTL AP,GU,13.4836,144.7961,77.4,20170312,71.06,87.08,80.06,4.473880,360.0,21.027236,0.0,0.0,0.0
1,ROOSEVELT ROADS,PR,18.2550,-65.6408,10.1,20170404,77.00,86.00,NaN,8.947760,360.0,23.040482,NaN,NaN,0.0
2,ROOSEVELT ROADS,PR,18.2550,-65.6408,10.1,20170420,NaN,NaN,NaN,8.500372,360.0,21.922012,NaN,NaN,0.0
3,SAN JUAN L M MARIN AP,PR,18.4325,-66.0108,2.7,20170120,69.08,82.04,NaN,3.355410,360.0,17.000744,0.0,0.0,0.0
4,SAN JUAN L M MARIN AP,PR,18.4325,-66.0108,2.7,20170217,73.04,87.08,NaN,4.697574,360.0,19.908766,0.0,0.0,0.0


In [72]:
data["date"] = pd.to_datetime(data["date"], format='%Y%m%d', errors='ignore')

# Make a state cut to select most of the midwest (neglect IL)
state_mask = (data["state"] == "MO") | (data["state"] == "SD") | (data["state"] == "NE") | (data["state"] == "KS") \
             | (data["state"] == "TX") | (data["state"] == "OK") | (data["state"] == "IA")
    
data_cut = data[state_mask].copy()

In [73]:
# Define "North Texas" as anything north of Austin, TX (source: Matt Wilde, Texan)
lat_cut = 30.2672 # Latitude of Austin, TX according to google
lat_mask = data_cut["latitude"] > lat_cut

# Processed dataframe!
df = data_cut[lat_mask].copy()
df.set_index("station", inplace=True)

In [74]:
df.head()

,state,latitude,longitude,elevation,date,TMIN,TMAX,TAVG,AWND,WDF5,WSF5,SNOW,SNWD,PRCP
station,,,,,,,,,,,,,,
BORGER HUTCHINSON CO AP,TX,35.695,-101.395,930.9,2017-01-15,32.00,33.08,NaN,6.710820,360.0,21.027236,NaN,0.0,0.838583
BORGER HUTCHINSON CO AP,TX,35.695,-101.395,930.9,2017-01-21,42.08,59.00,NaN,9.395148,360.0,38.027980,0.0,0.0,0.460630
BORGER HUTCHINSON CO AP,TX,35.695,-101.395,930.9,2017-01-22,35.96,55.94,NaN,11.855782,360.0,38.027980,NaN,0.0,0.051181
BORGER HUTCHINSON CO AP,TX,35.695,-101.395,930.9,2017-02-24,27.14,46.94,NaN,11.632088,360.0,29.080220,0.0,0.0,0.000000
BORGER HUTCHINSON CO AP,TX,35.695,-101.395,930.9,2017-03-01,27.14,55.94,NaN,7.381902,360.0,23.935258,0.0,0.0,0.000000


In [75]:
df.describe()

,latitude,longitude,elevation,TMIN,TMAX,TAVG,AWND,WDF5,WSF5,SNOW,SNWD,PRCP
count,35988.000000,35988.000000,35988.000000,35808.000000,35817.000000,12517.000000,34777.000000,34702.000000,34702.000000,23722.000000,24583.000000,35868.000000
mean,38.175513,-97.263839,480.508125,49.021999,72.754001,60.235769,9.509610,190.005907,27.255440,0.021514,0.245610,0.095080
std,4.177051,3.544738,359.690042,18.326581,19.706280,18.697304,4.144314,103.480399,9.173428,0.296099,1.721232,0.338079
min,30.320800,-106.375800,74.700000,-98.860000,-5.800000,-9.580000,0.447388,10.000000,4.921268,0.000000,0.000000,0.000000
25%,35.210000,-99.829700,249.300000,35.960000,60.980000,47.840000,6.487126,120.000000,21.027236,0.000000,0.000000,0.000000
50%,38.752500,-97.149400,350.500000,51.980000,78.080000,64.760000,8.947760,180.000000,25.948504,0.000000,0.000000,0.000000
75%,41.611700,-94.923300,594.700000,64.040000,87.080000,74.660000,12.079476,290.000000,33.106712,0.000000,0.000000,0.011811
max,45.668900,-89.570600,2100.100000,98.960000,111.920000,98.060000,99.096442,360.000000,170.007440,14.212606,29.015764,11.960636


In [76]:
grouped_means = df.groupby(df.index).mean()

In [77]:
grouped

,latitude,longitude,elevation,date,TMIN,TMAX,TAVG,AWND,WDF5,WSF5,SNOW,SNWD,PRCP
station,,,,,,,,,,,,,
ABERDEEN,45.4558,-98.4131,396.8,2.017050e+07,36.322077,61.319923,49.043448,10.092901,208.461538,26.980938,0.092520,0.649607,0.047093
ABILENE RGNL AP,32.4106,-99.6822,545.6,2.017050e+07,56.579692,80.926077,68.201379,10.859483,192.192308,28.562282,0.000000,0.000000,0.055633
ALLIANCE MUNI AP,42.0572,-102.8000,1196.0,2.017050e+07,35.030231,66.828615,NaN,10.190600,202.123552,29.505152,0.000000,0.000000,0.058692
ALTUS AFB,34.3622,-98.9761,386.2,2.017050e+07,54.694008,80.358125,NaN,11.234601,173.058824,29.281106,0.000000,0.000000,0.098880
AMARILLO,35.2333,-101.7089,1093.3,2.017050e+07,48.092692,76.973000,62.144828,12.831432,183.038462,32.357337,0.036493,0.043913,0.080875
ARLINGTON MUNI AP,32.6636,-97.0939,192.0,2.017050e+07,59.874286,81.292896,NaN,9.166272,187.076923,26.461279,0.000000,0.000000,0.097411
AUSTIN GREAT HILLS,30.4144,-97.7664,268.2,2.017050e+07,60.023409,82.991136,NaN,NaN,NaN,NaN,0.000000,0.000000,0.102108
AUSTIN-CAMP MABRY,30.3208,-97.7603,204.2,2.017050e+07,63.830231,85.948077,NaN,5.267994,173.346154,22.546634,0.000000,0.000000,0.101953
BARTLESVILLE F P FLD,36.7683,-96.0261,217.9,2.017050e+07,50.074769,76.042538,NaN,5.824648,144.461538,23.712424,0.000000,0.000000,0.151847
